# 1. Import Libraries & Helper Functions

## 1) library

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras import backend as K
from keras.layers import Input, Dense, Dropout, Input, Activation, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.models import Model, load_model, Sequential 

early_stopping = EarlyStopping(patience=10)

# Helper libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pandas as pd
from pandas import DataFrame as df

np.random.seed(777)

print(tf.__version__)

Using TensorFlow backend.


1.5.1


## 2) helper functions 

In [2]:
# prediction
def check_correct(predict, y):
    result = {}
    result['resistant-correct'] = 0
    result['resistant-wrong'] = 0
    result['sensitive-correct'] = 0
    result['sensitive-wrong'] = 0

    for i in range(len(predict)) :
        if predict[i] == y[i] :
            if y[i] == 0 :
                result['sensitive-correct'] += 1
            else :
                result['resistant-correct'] += 1
        else :
            if y[i] == 0 :
                result['sensitive-wrong'] += 1
            else :
                result['resistant-wrong'] += 1

    #for result_k, result_v in result.items():
    #    print(result_k +" : "+ str(result_v))
    sensitivity=result['resistant-correct']/(result['resistant-correct']+result['resistant-wrong'])
    specificity=result['sensitive-correct']/(result['sensitive-correct']+result['sensitive-wrong'])
    #print("Sensitivity :", sensitivity)
    #print("Specificity :", specificity)
    return sensitivity, specificity

In [3]:
# devide raw data into train / test & x_val / y_val
def data_split(raw_data, index_col, test_index):
    
    train_data = raw_data.iloc[list(raw_data.iloc[:,index_col]!=test_index)]
    test_data = raw_data.iloc[list(raw_data.iloc[:,index_col]==test_index)]
    
    y_val = train_data.Platinum_Status
    x_val = train_data.drop(["Platinum_Status","index"],axis=1)
    test_y_val = test_data.Platinum_Status
    test_x_val = test_data.drop(["Platinum_Status","index"],axis=1)
    
    return train_data, test_data, y_val, x_val, test_y_val, test_x_val

    # raw_data: have gene_expressions(maybe multiple columns), index column, Platinum_Status column.


In [4]:
# calculate all of model performance 
# - predictions(probability) / labeled predictions(0/1) / Loss / Accuracy / Sensitivity / Specificity / AUC values of Train / Test dataset.
# using trained models, or you can put predictions(probability) passively(in this case, Loss & Accuracy do not provided.)
def model_performance(information=False, Input_Prediction_Passively=False, using_model=None, tr_predictions=None, ts_predictions=None, tr_x_val=None, tr_y_val=None, ts_x_val=None, ts_y_val=None, output_list=None):
    
    if information == True:            
        print("options model_performance:\n1) using_model: keras models that you want to check performance. \"Input_Prediction_Passive\" option for input prediction list instead using models.\n3) tr_predictions & ts_predictions: prediction input passively. put this data only when not using keras model.\n4) tr_x_val & ts_x_val: input samples of train/test samples.\n4) tr_y_val & ts_y_val: results of train/test samples.\n5) output_list: return values that you want to recieve.\n CAUTION: Essential variable.\n\t tr_loss, tr_accuracy, tr_sensitivity, tr_specificity, tr_predictions, labeled_tr_predictions, tr_predictions_flat, roc_auc_tr,\nts_loss, ts_accuracy, ts_sensitivity, ts_specificity, ts_predictions, labeled_ts_predictions, ts_predictions_flat, roc_auc_ts,\nroc_auc_total\n\n* CAUTION: if 'None' value is returned, please check your input tr inputs(None value for tr outputs) or ts inputs(None value for ts outputs).") 
        return 0
    elif information != False:
        print("for using information options, please set 'information' variable for 'True'")
        return -1
    
    if using_model is None:
        if Input_Prediction_Passively == False:
            print("ERROR: There are no models for using.\nusing \"model_performance(information = True)\" for getting informations of this function.") 
            return -1
        elif (tr_predictions is None) and (ts_predictions is None): # No model/prediction input. no performance should be calculated.
                print("ERROR: Input prediction list instead using saved model.")
                return -1
        else: # No model input, but Input_Prediction_Passively is True & input prediction is valid.
            tr_loss,tr_accuracy= None, None
            ts_loss,ts_accuracy= None, None
            
    elif Input_Prediction_Passively == True: # both of model/prediction putted, could cause confusing.
        ch = input("You put both model and prediction. Select one method:\n'p' for using prediction only, 'm' using models only, 'n' for quit the function.")
        while 1:
            if ch == 'p':
                using_model = None
                break
            elif ch == 'm':
                tr_predictions = None
                ts_predictions = None
                break
            elif ch == 'e':
                return 0
            else:
                print("you put worng option: "+str(ch))
            ch = input("Select one method:\n'p' for using prediction only, 'm' using models only, 'n' for quit the function.")
                
    if output_list is None:
        print("ERROR: There are no output_list for return.\nusing \"model_performance(information = True)\" for getting informations of this function.")
        return -1
    
    if not(tr_x_val is None) and not(tr_y_val is None):
        # predict tr result only when no tr_prediction input
        if tr_predictions is None:
            tr_loss,tr_accuracy= using_model.evaluate(tr_x_val,tr_y_val)
            tr_predictions = using_model.predict(tr_x_val)
        # tr sensitivity / specificity
        labeled_tr_predictions = np.where(tr_predictions > 0.5, 1, 0).flatten()
        tr_sensitivity, tr_specificity = check_correct(labeled_tr_predictions, tr_y_val)
        tr_predictions_flat = tr_predictions[:,0]   
        # roc(tr)
        fpr_tr, tpr_tr, threshold_tr = metrics.roc_curve(tr_y_val, tr_predictions)
        roc_auc_tr = metrics.auc(fpr_tr, tpr_tr)
    
    if not(ts_x_val is None) and not(ts_y_val is None):
        # predict ts result only when no ts_prediction input
        if ts_predictions is None:
            ts_loss,ts_accuracy= using_model.evaluate(ts_x_val,ts_y_val)
            ts_predictions = using_model.predict(ts_x_val)
        labeled_ts_predictions = np.where(ts_predictions > 0.5, 1, 0).flatten()
        ts_sensitivity, ts_specificity = check_correct(labeled_ts_predictions, ts_y_val)
        ts_predictions_flat = ts_predictions[:,0]   
        # roc(ts)
        fpr_ts, tpr_ts, threshold_ts = metrics.roc_curve(ts_y_val, ts_predictions)
        roc_auc_ts = metrics.auc(fpr_ts, tpr_ts)    
    
    if (not(tr_x_val is None) and not(tr_y_val is None)) and (not(ts_x_val is None) and not(ts_y_val is None)):
        y_true = np.append(tr_y_val, ts_y_val)
        y_pred = np.append(tr_predictions, ts_predictions)
        fpr_total, tpr_total, threshold_total = metrics.roc_curve(y_true, y_pred)
        roc_auc_total = metrics.auc(fpr_total, tpr_total)
        
        
    return_list = []
    
    for output in output_list:
        
        if(output == "tr_loss"):
            return_list.append(tr_loss)
                               
        elif(output == "tr_accuracy"):
            return_list.append(tr_accuracy)
                               
        elif(output == "tr_sensitivity"):
            return_list.append(tr_sensitivity)
                               
        elif(output == "tr_specificity"):
            return_list.append(tr_specificity)
                               
        elif(output == "tr_predictions"):
            return_list.append(tr_predictions)
                               
        elif(output == "labeled_tr_predictions"):
            return_list.append(labeled_tr_predictions)
                               
        elif(output == "tr_predictions_flat"):
            return_list.append(tr_predictions_flat)
            
        elif(output == "roc_auc_tr"):
            return_list.append(roc_auc_tr)

        elif(output == "ts_loss"):
            return_list.append(ts_loss)
                               
        elif(output == "ts_accuracy"):
            return_list.append(ts_accuracy)
                               
        elif(output == "ts_sensitivity"):
            return_list.append(ts_sensitivity)
                               
        elif(output == "ts_specificity"):
            return_list.append(ts_specificity)
                               
        elif(output == "ts_predictions"):
            return_list.append(ts_predictions)
                               
        elif(output == "labeled_ts_predictions"):
            return_list.append(labeled_ts_predictions)
                               
        elif(output == "ts_predictions_flat"):
            return_list.append(ts_predictions_flat)
        
        elif(output == "roc_auc_ts"):
            return_list.append(roc_auc_ts)
            
        elif(output == "roc_auc_total"):
            return_list.append(roc_auc_total)
                               
        else:
            print("There are no options <"+str(output)+">. Please refer these output options:\ntr_loss, tr_accuracy, tr_sensitivity, tr_specificity, tr_predictions, labeled_tr_predictions, tr_predictions_flat, roc_auc_tr,\nts_loss, ts_accuracy, ts_sensitivity, ts_specificity, ts_predictions, labeled_ts_predictions, ts_predictions_flat, roc_auc_ts,\nroc_auc_total")
    
    return return_list

# 2. Preparation: Load Data Files, Select the Type for Optimizing

In [6]:
types = ["OV_six_fold_Annotation3000_400", 
         "OV_six_fold_CV_400", 
         "OV_six_fold_Var_400", "OV_six_fold_new_Diff_400",
         "OV_six_fold_Clin", 
         "OV_six_fold_SNV" 
         ]

# input pathes
path = "../TC_six_fold_subsamples/"
save_model_path = "../models/"
save_prediction_path = "../prediction_result/"
save_result_path = "../result/"

file_1 = path+types[0]+".csv"
file_2 = path+types[1]+".csv"
file_3 = path+types[2]+".csv"
file_4 = path+types[3]+".csv"
file_5 = path+types[4]+".csv"
file_6 = path+types[5]+".csv"

idx_col = 0

data_1 = pd.read_csv(file_1,index_col=idx_col)
data_2 = pd.read_csv(file_2,index_col=idx_col)
data_3 = pd.read_csv(file_3,index_col=idx_col)
data_4 = pd.read_csv(file_4,index_col=idx_col)
data_5 = pd.read_csv(file_5,index_col=idx_col)
data_6 = pd.read_csv(file_6,index_col=idx_col)

sample_1,features_1 = data_1.shape
sample_2,features_2 = data_2.shape
sample_3,features_3 = data_3.shape
sample_4,features_4 = data_4.shape
sample_5,features_5 = data_5.shape
sample_6,features_6 = data_6.shape

# Data frame include index & Platinum_Status column, substract 2 to calculate real number of features 
[features_1, features_2, features_3, features_4, features_5, features_6] = [features_1-2, features_2-2, features_3-2, features_4-2, features_5-2, features_6-2]

ds_list = [data_1, data_2, data_3, data_4, data_5, data_6]
sam_list = [sample_1, sample_2, sample_3, sample_4, sample_5, sample_6]
fea_list = [features_1, features_2, features_3, features_4, features_5, features_6]

print("[1] file_name: ", types[0], "\nsample : {}  \nfeatures : {}".format(sample_1,features_1))
print("[2] file_name: ", types[1], "\nsample : {}  \nfeatures : {}".format(sample_2,features_2))
print("[3] file_name: ", types[2], "\nsample : {}  \nfeatures : {}".format(sample_3,features_3))
print("[4] file_name: ", types[3], "\nsample : {}  \nfeatures : {}".format(sample_4,features_4))
print("[5] file_name: ", types[4], "\nsample : {}  \nfeatures : {}".format(sample_5,features_5))
print("[6] file_name: ", types[5], "\nsample : {}  \nfeatures : {}".format(sample_6,features_6))

print("\ndata files read.\n")

[1] file_name:  OV_six_fold_Annotation3000_400 
sample : 217  
features : 400
[2] file_name:  OV_six_fold_CV_400 
sample : 217  
features : 400
[3] file_name:  OV_six_fold_Var_400 
sample : 217  
features : 400
[4] file_name:  OV_six_fold_new_Diff_400 
sample : 217  
features : 400
[5] file_name:  OV_six_fold_Clin 
sample : 287  
features : 35
[6] file_name:  OV_six_fold_SNV 
sample : 213  
features : 13814

data files read.



In [13]:
print(types)
ch = input("select types: ")
select_type = types[int(ch)]
select_data = ds_list[int(ch)]
select_sample = sam_list[int(ch)]
select_features = fea_list[int(ch)]

print("\nselect: "+select_type )

dataset = {"tr_data":[], "ts_data":[], "tr_y_val":[], "tr_x_val":[], "ts_y_val":[], "ts_x_val":[]}
val_name = ["tr_data", "ts_data", "tr_y_val", "tr_x_val", "ts_y_val", "ts_x_val"]

for ts_i in range(1, 6):
    tr_data, ts_data, tr_y_val, tr_x_val, ts_y_val, ts_x_val = data_split(raw_data = select_data, index_col = -1, test_index = ts_i)
    dataset['tr_data'].append(tr_data)
    dataset['ts_data'].append(ts_data)
    dataset['tr_x_val'].append(tr_x_val)
    dataset['tr_y_val'].append(tr_y_val)
    dataset['ts_x_val'].append(ts_x_val)
    dataset['ts_y_val'].append(ts_y_val)      

print("data files splited with five fold.\n")
print("n of samples: ")
print("[1]: "+str(len(dataset['tr_x_val'][0]))+" for train, "+str(len(dataset['ts_x_val'][0]))+" for test.")
print("[2]: "+str(len(dataset['tr_x_val'][1]))+" for train, "+str(len(dataset['ts_x_val'][1]))+" for test.")
print("[3]: "+str(len(dataset['tr_x_val'][2]))+" for train, "+str(len(dataset['ts_x_val'][2]))+" for test.")
print("[4]: "+str(len(dataset['tr_x_val'][3]))+" for train, "+str(len(dataset['ts_x_val'][3]))+" for test.")
print("[5]: "+str(len(dataset['tr_x_val'][4]))+" for train, "+str(len(dataset['ts_x_val'][4]))+" for test.")

['OV_six_fold_Annotation3000_400', 'OV_six_fold_CV_400', 'OV_six_fold_Var_400', 'OV_six_fold_new_Diff_400', 'OV_six_fold_Clin', 'OV_six_fold_SNV']
select types: 3

select: OV_six_fold_new_Diff_400
data files splited with five fold.

n of samples: 
[1]: 186 for train, 31 for test.
[2]: 186 for train, 31 for test.
[3]: 186 for train, 31 for test.
[4]: 187 for train, 30 for test.
[5]: 187 for train, 30 for test.


In [14]:
# i is variable that indicate fold index number.
i = input("test index(1~5): ")
i = int(i)

model_list = []
model_num_list = []
test_index_list = []
tr_acc_list = []
ts_acc_list = []
tr_sensitivity_list = []
ts_sensitivity_list = []
tr_specificity_list = []
ts_specificity_list = []
tr_auc_list = []
ts_auc_list = []
tot_auc_list = []

k = 0
lr_box = []
layers_box = []
batch_size_box = []
drop_out_box = []
input_drop_out_box = []
batch_normalize_box = []

test index(1~5): 1


# 3. Build & Evaluate Model

In [17]:
# parameter change

lr = 0.01
drop_out_m = 0
input_drop_out_m = 0.3
batch_size = 5
BN = True
layers = [100, 50]

In [18]:
adam = optimizers.Adam(lr=lr)

m_tr_loss_best = 100 # for saving best loss value 
best_m_model=[] #for saving best model
count=0 # for early stopping

# 2) model build
input_m = Input(shape=(select_features,))
m_m_dp = Dropout(input_drop_out_m)(input_m)
for l in layers:
    if BN == True:
        m_m = Dense(l)(m_m_dp)
        m_m_bn = BatchNormalization(axis=1, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones')(m_m)
        m_m_dp = Activation("relu")(m_m_bn)
    else:
        m_m = Dense(l,activation='relu')(m_m_dp)
        m_m_dp = Dropout(drop_out_m)(m_m)

m_m_final = m_m_dp
output_m = Dense(1, activation="sigmoid")(m_m_final)
m_model = Model(inputs=input_m,outputs=output_m)
m_model.compile(optimizer=adam, 
                loss='binary_crossentropy',
                metrics=['accuracy'])

# 3) Training: if no increase of tr_loss three times, stop training.
while 1:
    m_model.fit(dataset['tr_x_val'][i], dataset['tr_y_val'][i], batch_size=batch_size, nb_epoch=1, verbose = 0)
    m_tr_loss=m_model.evaluate( dataset['tr_x_val'][i], dataset['tr_y_val'][i])[0]
    if m_tr_loss < m_tr_loss_best: # new best model. count reset.
        m_tr_loss_best = m_tr_loss
        count=0
        best_m_model = m_model
    if count>10: # no increase three time. stop.
        m_model = best_m_model
        break
    else: count=count+1
print("Model " + ch+"-"+str(i)+"_"+str(k)+" trained.")

# 4) save model
#model_list.append(m_model)
m_model.save(save_model_path+"/m_"+ch+"-"+str(i)+"_"+str(k)+".h5")
print("Model "+ch+"-"+str(i)+"_"+str(k)+" saved.")

# 5) evaluate model
m_output_list = model_performance(
    information = False, using_model=m_model,Input_Prediction_Passively = False, 
    tr_x_val=dataset['tr_x_val'][i], tr_y_val=dataset['tr_y_val'][i], ts_x_val=dataset['ts_x_val'][i], ts_y_val=dataset['ts_y_val'][i],
    output_list=["tr_loss", "tr_accuracy", "tr_sensitivity", "tr_specificity", "tr_predictions",
                 "labeled_tr_predictions", "tr_predictions_flat", "roc_auc_tr", 
                 "ts_loss", "ts_accuracy", "ts_sensitivity", "ts_specificity", "ts_predictions",
                 "labeled_ts_predictions", "ts_predictions_flat", "roc_auc_ts", 
                 "roc_auc_total"])

m_tr_loss, m_tr_accuracy, m_tr_sensitivity, m_tr_specificity, m_tr_predictions, m_labeled_tr_predictions, m_tr_predictions_flat, m_roc_auc_tr, m_ts_loss, m_ts_accuracy, m_ts_sensitivity, m_ts_specificity, m_ts_predictions,m_labeled_ts_predictions, m_ts_predictions_flat, m_roc_auc_ts, m_roc_auc_total = m_output_list

print("Overall AUC: ", m_roc_auc_total)
print("Train AUC: ", m_roc_auc_tr)
print("Test AUC: ", m_roc_auc_ts)

print("Train Accuracy: {}".format(m_tr_accuracy))
print("Train Sensitivities & Specificities : "+str(m_tr_sensitivity)+", "+str(m_tr_specificity))
print("Test Accuracy: {}".format(m_ts_accuracy))
print("Test Sensitivities & Specificities : "+str(m_ts_sensitivity)+", "+str(m_ts_specificity))

lr_box.append(lr)
layers_box.append(layers)
batch_size_box.append(batch_size)
drop_out_box.append(drop_out_m)
input_drop_out_box.append(input_drop_out_m)
batch_normalize_box.append(BN)
model_num_list.append(ch)
test_index_list.append(i)
tr_acc_list.append(m_tr_accuracy)
ts_acc_list.append(m_ts_accuracy)
tr_sensitivity_list.append(m_tr_sensitivity)
ts_sensitivity_list.append(m_ts_sensitivity)
tr_specificity_list.append(m_tr_specificity)
ts_specificity_list.append(m_ts_specificity)
tr_auc_list.append(m_roc_auc_tr)
ts_auc_list.append(m_roc_auc_ts)
tot_auc_list.append(m_roc_auc_total)

# save prediction result.

tr_df_m = pd.DataFrame(data={"patient":list(dataset['tr_data'][i].index), "hypothesis 1": list(m_tr_predictions_flat), 
                        "prediction":list(m_labeled_tr_predictions), "Platinum_Status":list(dataset['tr_y_val'][i])}) 
tr_df_m.to_csv(save_prediction_path+"m_"+ch+"-"+str(i)+"_"+str(k)+"_tr.csv", index=False, header=True, columns = ["patient", "hypothesis 1", "prediction", "Platinum_Status"])

ts_df_m = pd.DataFrame(data={"patient":list(dataset['ts_data'][i].index), "hypothesis 1": list(m_ts_predictions_flat), 
                        "prediction":list(m_labeled_ts_predictions), "Platinum_Status":list(dataset['ts_y_val'][i])})
ts_df_m.to_csv(save_prediction_path+"m_"+ch+"-"+str(i)+"_"+str(k)+"_ts.csv", index=False, header=True, columns = ["patient", "hypothesis 1", "prediction", "Platinum_Status"])
k = k+1

c:\users\hgh97\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:28: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


186/186 [==============================] - 0s 129us/step
Model 3-1_1 trained.
Model 3-1_1 saved.
31/31 [==============================] - 0s 97us/step
Overall AUC:  0.992166563595135
Train AUC:  1.0
Test AUC:  0.8737373737373737
Train Accuracy: 1.0
Train Sensitivities & Specificities : 1.0, 1.0
Test Accuracy: 0.9032257795333862
Test Sensitivities & Specificities : 0.7777777777777778, 0.9545454545454546


In [12]:
df_1 = df(data = {'model_numbers':model_num_list, 
                'index': range(0,k),
                'rate':lr_box,
                'layers':layers_box,
                'batch_size': batch_size_box,
                'input_drop_out':input_drop_out_box,
                'drop_out':drop_out_box,
                'batch_normalize':batch_normalize_box,
                'test_index':test_index_list ,
                'tr_accuracy':tr_acc_list, 
                'tr_sensitivity':tr_sensitivity_list, 
                'tr_specificity':tr_specificity_list, 
                'ts_accuracy': ts_acc_list,
                'ts_sensitivity':ts_sensitivity_list, 
                'ts_specificity':ts_specificity_list, 
                "tr_auc":tr_auc_list, 
                "ts_auc":ts_auc_list, 
                "total_auc":tot_auc_list}, 
          columns =['index','model_numbers', 'test_index', 'rate', 'layers','batch_size','input_drop_out','drop_out','batch_normalize', 'tr_accuracy', 'tr_sensitivity', 'tr_specificity', 'ts_accuracy', 'ts_sensitivity', 'ts_specificity', "tr_auc", "ts_auc", "total_auc"])
df_1.to_csv(save_result_path+select_type+"_"+str(i)+".csv", index=False)

ValueError: arrays must all be same length